In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
path_to_images = "../Datasets/test_images"

In [ ]:
## iterating via directory
qr_df = pd.read_csv(path_to_images+"/test_qr_labels.csv")

In [ ]:
from utils import get_test_transform
from QRDatasets import QRDatasets
# qr_df['x'] = -1
# qr_df['y'] = -1
# qr_df['w'] = -1
# qr_df['h'] = -1
test_tf = get_test_transform()
test_dataset = QRDatasets(path_to_images, qr_df, transforms=test_tf)

In [ ]:
# for i in range(test_dataset.__len__()):
#     print(i)
#     img, target, idx = test_dataset.__getitem__(i)

In [ ]:
!ls models/

In [ ]:
import os
import torch
with open("models/best_model", 'rb') as f:
    model = torch.load("models/best_model")

In [ ]:
from torch.utils.data import DataLoader
from utils import collate_fn
data_loader = DataLoader(test_dataset, shuffle=True, batch_size=4,pin_memory=True, collate_fn=collate_fn, num_workers=4)

In [ ]:
import matplotlib.pyplot as plt
for images, targets, image_ids in data_loader:
    patch_size = image_ids.__len__()
    for i in range(patch_size):
        model.eval()
        output = model([images[i].cuda()])
#         print(output)
        
        # converting bbox location into range (0, 1)
        boxes = output[0]['boxes']/512
        scores = output[0]['scores']
        print(output)
        
        # obtaining the original images
        idx = image_ids[i]
        records = qr_df[qr_df.index == idx]
        img_path = path_to_images + "/" + str(records["Location"].values[0]) + "/" + \
                   str(records["Camera"].values[0]) + "/" + str(records["File Name"].values[0])
        img = cv2.imread(img_path)
        
        # fitting bbox location in the original images
        for j in range(boxes.shape[0]):
            score = scores[j].item()
            if score < 0.5:
                continue
              
            box = boxes[j]
            x1 = int(box[0] * img.shape[1])
            y1 = int(box[1] * img.shape[0])
            x2 = int(box[2] * img.shape[1])
            y2 = int(box[3] * img.shape[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0),10)
        
        plt.imshow(img)
        plt.show()